# Exercise 11- RNA-Protein Interaction with Transformers

## Overview

In this exercise, we will download the pre-trained Bidirectional Encoder Representations from Transformers model for DNA (DNABERT) and fine-tune it to predict the RNA-binding sites of the RNA-binding protein, PUM2. We will then also implement an LSTM model to train and evaluate it on the same dataset as a comparison.

This notebook involves the following steps:
1. [Preprocess Data](#preprocess-data)
2. [Fine-tune DNABERT](#fine-tune-dnabert)
3. [Implement and Train RNN](#implement-and-train-rnn)

## Preprocess Data

In the link, you will find 3101 positive and 3101 negative RNA-binding sites for the RNA-binding protein, PUM2. Each file consists of a header followed by an RNA sequence ie. an RNA known to bind with PUM2 (positive) or that does not bind to PUM2 (negative)

We will start by preprocessing this dataset to be used by DNABERT and our RNN. Since this is a binary classification problem, we will define X as the sequence and corresponding label y as whether this sequence binds to PUM2 (1) or not (0), depending on which file it comes from, such that all sequences in `positives.fa` is assigned a label of `1` and all sequences in `negatives.fa` is assigned a lable of `0`. We must also convert all RNA sequences to DNA sequences to be processed by DNABERT by changing all instances of `U` to `T`. Finally, we must create overlapping k-mers of these sequences to be in the format expected by DNABERT.

Download DNABERT from its repository: https://github.com/jerryji1993/DNABERT

In [1]:
# DNABERT requires python3.7 and must create a conda env to install it then set as kernel for this notebook before starting

# %conda create -n dnabert python=3.7
# %conda activate dnabert
# %conda install pytorch torchvision cudatoolkit=10.0 -c pytorch

# Clone repo and set up BERT locally 
# %git clone https://github.com/jerryji1993/DNABERT
# %cd DNABERT
# %python3 -m pip install --editable .
# %cd examples
# %python3 -m pip install -r requirements.txt

In [2]:
# Install required packages 
%pip install transformers pandas numpy pytoch scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [1]:
%%bash
python --version

Python 3.7.12


In [2]:
# Import libraries
import os
import re
import pandas as pd
import sys
import pandas as pd
import numpy as np
from itertools import product

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn import Linear

import sklearn
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold

/home/koeksalr/miniconda3/envs/dnabert2/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Why k-mers?

DNABERT is based on BERT, originally designed for Natural Language Processing (NLP), adapted for DNA language. Traditional language models like BERT requires the input as discrete symbols and thus requires tokenization to convert input into meaningful groups to learn embeddings on and then help attention to learn. 

DNABERT uses the tokenization startegy of breaking input up into overlapping k-mers. This is in line with biological context and relevance where several biological motifs are made up of these groups within nucleotide sequences, e.g. `TATA` boxes in promoters, `ATG` start codons in Open Reading Frames (ORFs), `AGGT` for splicing signals, etc. This tokenization strategy therefore, allows the model to differentiate between and learn embeddings on these biological motifs, which can then be passed to the attention head to learn meaningful relationships between these motifs. If we consider the case of tokenizing individual sequences then each sequence, sometimes extremely long for DNA, will be considered unique and thus no relationships can be found between more meaningful motifs. In the other extreme, if we tokenize individual nucleotides then these motifs and the biological context also retained from them being overlapping, then this biologically significant contextual information would be lost.

For example if we consider the sequence, `ATGGAT`, with an overlapping 3-mer split, we get the tokens `ATG`, `TGG`, `GGA`, and `GAT`. If these were non-overlapping, would get the tokens `ATG` and `GAT` which could miss potential motifs and thus the attention head will not be able to learn any potential relationshios between those with other tokens. If we tokenized this using single-nucleotide tokenization, we get the tokens `A`, `T`, `G`, `G`, `A`, and `T`. Also note that with kmers of length `L`, we get `L - k + 1` tokens and thus nearly the same number of tokens. However, the key difference is that we now obtain much more meaningful and biologically significant groups that the attention head can learn relationships between. With this strategy e.g. the attention head would not be able to differentiate between the `ATG` and `GAT` but could only find relationships between the individual letters, e.g. `A` from `ATG` and `G` from `GAT`, which would be no different to it than `A` and `G` in `ATG`, etc. 

This would therefore be analogous to a language model trying to find relationships between individual letters in a text instead of words when trying to determine the meaning of a text.

In [14]:
def read_fasta_into_dic(fasta_file,
                        convert_to_rna=False,
                        convert_to_dna=True,
                        all_uc=True,
                        skip_n_seqs=False):
    """
    Read in FASTA sequences from file, return dictionary with mapping:
    sequence_id -> sequence

    convert_to_rna:
        Convert input sequences to RNA.
    all_uc:
        Convert all sequence characters to uppercase.
    skip_n_seqs:
        Skip sequences that contain N letters.

    """

    assert os.path.exists(fasta_file), "Given FASTA file \"%s\" not found" %(fasta_file)

    seqs_dic = {}
    seq_id = ""

    with open(fasta_file) as f:
        for line in f:
            if re.search(">.+", line):
                m = re.search(">(.+)", line)
                seq_id = m.group(1)
                assert seq_id not in seqs_dic, "non-unique FASTA header \"%s\" in \"%s\"" % (seq_id, fasta_file)
                seqs_dic[seq_id] = ""
            elif re.search("[ACGTUN]+", line, re.I):
                m = re.search("([ACGTUN]+)", line, re.I)
                if seq_id in seqs_dic:
                    '''if convert_to_rna:
                        seqs_dic[seq_id] += m.group(1).replace("T","U").replace("t","u")'''
                    if convert_to_dna:
                        seqs_dic[seq_id] += m.group(1).replace("U","T").replace("u","t")
                    else:
                        seqs_dic[seq_id] += m.group(1)
                if all_uc:
                    seqs_dic[seq_id] = seqs_dic[seq_id].upper()
    f.closed

    assert seqs_dic, "no sequences read in (input FASTA file \"%s\" empty or mal-formatted?)" %(fasta_file)

    # If sequences with N nucleotides should be skipped.
    c_skipped_n_ids = 0
    if skip_n_seqs:
        del_ids = []
        for seq_id in seqs_dic:
            seq = seqs_dic[seq_id]
            if re.search("N", seq, re.I):
                print ("WARNING: sequence with seq_id \"%s\" in file \"%s\" contains N nucleotides. Discarding sequence ... " % (seq_id, fasta_file))
                c_skipped_n_ids += 1
                del_ids.append(seq_id)
        for seq_id in del_ids:
            del seqs_dic[seq_id]
        assert seqs_dic, "no sequences remaining after deleting N containing sequences (input FASTA file \"%s\")" %(fasta_file)
        if c_skipped_n_ids:
            print("# of N-containing sequences discarded:  %i" %(c_skipped_n_ids))

    return seqs_dic


def seq2kmer(seq, k):
    """
    Convert original sequence to kmers
    
    Arguments:
    seq -- str, original sequence.
    k -- int, kmer of length k specified.
    
    Returns:
    kmers -- str, kmers separated by space

    """
    kmer = [seq[x:x+k] for x in range(len(seq)+1-k)]
    kmers = " ".join(kmer)
    return kmers


def read_seqs(seq_dict, label, k=3, columns=["sequence", "label"]):
    seqs_kmer = list()
    class_label = list()
    for item in seq_dict:
        seq = seq_dict[item]
        k_seq = seq2kmer(seq, k)
        seqs_kmer.append(k_seq)
        class_label.append(label)
    dataframe = pd.DataFrame(zip(seqs_kmer, class_label), columns=columns, index=None)
    return dataframe

def merge_pos_neg_kfold(pos_seqs, neg_seqs, n_splits=5, k=3, test_train=0.8):
    """
    Merge positive and negative sequences into one DataFrame
    Return a list of (train_df, test_df) folds for cross-validation
    """
    # Read sequences -> k-mers
    pos_df = read_seqs(pos_seqs, 1, k=k)
    neg_df = read_seqs(neg_seqs, 0, k=k)

    # Combine and shuffle
    merged_df = pd.concat([pos_df, neg_df], axis=0)
    merged_df = merged_df.sample(frac=1, random_state=42).reset_index(drop=True)

    # If 0 folds provided then do not create fold splits but use full train and test data
    if n_splits == 0:
        split_pos = int(test_train * len(merged_df.index))
        train = merged_df[:split_pos]
        test = merged_df.drop(train.index)
        train.to_csv("./data/train.tsv", sep="\t", index=None)
        test.to_csv("./data/test.tsv", sep="\t", index=None)
        print(merged_df)
        print()
        print(train)
        print()
        print(test)
    else:
        # Extract features/labels for StratifiedKFold
        X = merged_df["sequence"]
        y = merged_df["label"]
        
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

        folds = []
        for fold_idx, (train_index, test_index) in enumerate(skf.split(X, y)):
            train_df = merged_df.iloc[train_index].reset_index(drop=True)
            test_df = merged_df.iloc[test_index].reset_index(drop=True)
            folds.append( (train_df, test_df) )

            # Optionally save to disk for each fold
            train_df.to_csv(f"./data/train_fold{fold_idx+1}.tsv", sep="\t", index=False)
            test_df.to_csv(f"./data/test_fold{fold_idx+1}.tsv", sep="\t", index=False)

        return folds

In [ ]:
pos_fasta = "./data/positives.fa"
neg_fasta = "./data/negatives.fa"

# Load FASTA sequences into dictionaries.
pos_seqs_dic = read_fasta_into_dic(pos_fasta, 
                                    convert_to_rna=True, 
                                    all_uc=True,
                                    skip_n_seqs=True)
neg_seqs_dic = read_fasta_into_dic(neg_fasta,
                                    convert_to_rna=True, 
                                    all_uc=True,
                                    skip_n_seqs=True)

print("# positive sequences:  %i" %(len(pos_seqs_dic)))
print("# negative sequences:  %i" %(len(neg_seqs_dic)))

# Split train and test data into 5 folds
folds = merge_pos_neg_kfold(pos_seqs_dic, neg_seqs_dic, n_splits=5)

for i, (train_df, test_df) in enumerate(folds):
    print(f"Fold {i+1}")
    print("Train shape:", train_df.shape)
    print("Test shape:", test_df.shape)
    print(train_df)

# positive sequences:  3101
# negative sequences:  3101
Fold 1
Train shape: (4961, 2)
Test shape: (1241, 2)
                                               sequence  label
0     TGT GTC TCT CTT TTT TTT TTA TAT ATC TCA CAT AT...      0
1     CTG TGT GTC TCA CAT ATA TAG AGC GCC CCT CTC TC...      1
2     GAA AAA AAC ACC CCT CTG TGC GCC CCT CTG TGT GT...      1
3     ATC TCC CCT CTG TGG GGG GGC GCT CTT TTC TCC CC...      0
4     AAA AAT ATC TCA CAT ATC TCT CTG TGT GTG TGT GT...      1
...                                                 ...    ...
4956  TCT CTC TCG CGG GGC GCT CTG TGC GCC CCG CGG GG...      0
4957  TGT GTT TTG TGT GTA TAA AAT ATT TTT TTT TTT TT...      0
4958  CTA TAG AGT GTG TGC GCC CCT CTG TGG GGG GGA GA...      0
4959  AGG GGT GTA TAA AAA AAA AAG AGT GTT TTT TTT TT...      0
4960  AAA AAT ATT TTA TAA AAT ATT TTT TTG TGC GCT CT...      1

[4961 rows x 2 columns]
Fold 2
Train shape: (4961, 2)
Test shape: (1241, 2)
                                               sequence  la

## Fine-tune DNABERT

Next we will download a pre-trained version of DNABERT and set it up locally to fine-tune it on our data.

Download the pre-trained DNABERT3 from the following link and unzip it: https://drive.google.com/file/d/1nVBaIoiJpnwQxiz4dSq6Sv9kBKfXhZuM/view

In [6]:
!unzip ./dnabert3/3-new-12w-0.zip -d .

Archive:  ./dnabert3/3-new-12w-0.zip
replace ./3-new-12w-0/vocab.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


The following shell file will run `finetune.py` to fine-tune the pre-trained model we downloaded. The shell file contains all the parameters needed to fine-tune DNABERT, including the path to the pre-trained model downloaded previously and our data we preprocessed. Modify the paths to the files as needed.

In [ ]:
%%bash
cd DNABERT/examples

export KMER=3
export MODEL_PATH='/home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/dnabert3/3-new-12w-0'
export DATA_PATH='/home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/data'
export OUTPUT_PATH='/home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/output'

# Run the finetuning 5 times for 5-fold cross-validation
for i in 1 2 3 4 5
do
    echo "=== Starting training for fold $i ==="

    # Copy the correct fold's train & test files into DNABERT's train/test filenames
    cp $DATA_PATH/train_fold${i}.tsv $DATA_PATH/train.tsv
    cp $DATA_PATH/test_fold${i}.tsv  $DATA_PATH/test.tsv

    # Run DNABERT
    python run_finetune.py \
        --model_type dna \
        --tokenizer_name=dna$KMER \
        --model_name_or_path $MODEL_PATH \
        --task_name dnaprom \
        --do_train \
        --do_eval \
        --data_dir $DATA_PATH \
        --max_seq_length 100 \
        --per_gpu_eval_batch_size=32 \
        --per_gpu_train_batch_size=32 \
        --learning_rate 2e-4 \
        --num_train_epochs 5.0 \
        --output_dir $OUTPUT_PATH/fold_${i} \
        --evaluate_during_training \
        --logging_steps 100 \
        --save_steps 4000 \
        --warmup_percent 0.1 \
        --hidden_dropout_prob 0.1 \
        --overwrite_output \
        --weight_decay 0.01 \
        --n_process 8
done

echo "=== All folds completed! ==="



Optionally, you may instead uncomment the cell below to run the same fine-tuning script for DNABERT on the full dataset (without cross-validation).

In [ ]:
# %%bash 
# cd DNABERT/examples

# export KMER=3
# export MODEL_PATH='/home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/dnabert3/3-new-12w-0'
# export DATA_PATH='/home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/data'
# export OUTPUT_PATH='/home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/output'

# python run_finetune.py \
#     --model_type dna \
#     --tokenizer_name=dna$KMER \
#     --model_name_or_path $MODEL_PATH \
#     --task_name dnaprom \
#     --do_train \
#     --do_eval \
#     --data_dir $DATA_PATH \
#     --max_seq_length 100 \
#     --per_gpu_eval_batch_size=32   \
#     --per_gpu_train_batch_size=32   \
#     --learning_rate 2e-4 \
#     --num_train_epochs 5.0 \
#     --output_dir $OUTPUT_PATH \
#     --evaluate_during_training \
#     --logging_steps 100 \
#     --save_steps 4000 \
#     --warmup_percent 0.1 \
#     --hidden_dropout_prob 0.1 \
#     --overwrite_output \
#     --weight_decay 0.01 \
#     --n_process 8

<class 'transformers.tokenization_dna.DNATokenizer'>
{"eval_acc": 0.9008863819500403, "eval_f1": 0.9008861245249425, "eval_mcc": 0.8028972501166461, "eval_auc": 0.9608326186949403, "eval_precision": 0.901480969361764, "eval_recall": 0.9014162833606195, "learning_rate": 0.00019373219373219373, "loss": 0.5076427334547042, "step": 100}
{"eval_acc": 0.9016921837228042, "eval_f1": 0.9016308020167368, "eval_mcc": 0.8032662931269479, "eval_auc": 0.9572646241002053, "eval_precision": 0.9016655057336231, "eval_recall": 0.9016007900002598, "learning_rate": 0.00016524216524216523, "loss": 0.2715525978431106, "step": 200}
{"eval_acc": 0.9145850120870266, "eval_f1": 0.9142549108562905, "eval_mcc": 0.8312696990541014, "eval_auc": 0.9699553026168758, "eval_precision": 0.9176453764490415, "eval_recall": 0.9136340011953952, "learning_rate": 0.00013675213675213676, "loss": 0.25035233695060016, "step": 300}
{"eval_acc": 0.9121676067687349, "eval_f1": 0.911887262804842, "eval_mcc": 0.8257795050944344, "ev

01/24/2025 17:11:21 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
01/24/2025 17:11:21 - INFO - transformers.configuration_utils -   loading configuration file /home/koeksalr/Downloads/ml_in_life_sciences/ML_LS_resources/exercise_11_rna_protein_transformer_hands_on/dnabert3/3-new-12w-0/config.json
01/24/2025 17:11:21 - INFO - transformers.configuration_utils -   Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "do_sample": false,
  "eos_token_ids": 0,
  "finetuning_task": "dnaprom",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_p

## Implement and Train RNN 

We will implement and set up local RNN (LSTM) to compare against DNABERT. We will implement an LSTM-based sequence classifier for this with the following architecture:

1) Embedding
2) LSTM
3) FC hidden
4) ReLU
5) Linear 
6) Sigmoid

RNNSeqClassifier(
  (embedding): Embedding(4097, 128)
  (lstm): LSTM(128, 100, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (fc_hidden): Linear(in_features=200, out_features=64, bias=True)
  (fc_relu): ReLU()
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()

In [18]:

train_path = "./data/train.tsv"
test_path = "./data/test.tsv"
vocab = "AGCT"
batch_size = 32
hidden_dim = 100
fc_hidden_dim = 64
n_layers = 2
learning_rate = 2e-4
n_epochs = 50
dropout = 0.1
embedding_dim = 128
bidirectional = True
vocab_size = 4096 + 1
output_dim = 1


class RNNSeqClassifier(torch.nn.Module):
    
    def __init__(self):
        
        super(RNNSeqClassifier, self).__init__()
        # embedding layer: vector representation of each kmer index
        self.embedding = torch.nn.Embedding(vocab_size, embedding_dim)
        # LSTM: recurrent layer that learns connections
        self.lstm = torch.nn.LSTM(embedding_dim,
            hidden_dim,
            num_layers = n_layers,
            bidirectional = bidirectional,
            dropout = dropout,
            batch_first = True
        )
        # dense layer for assigning class to each sequence of kmers
        self.fc_hidden = torch.nn.Linear(hidden_dim * 2, fc_hidden_dim)
        self.fc_relu = torch.nn.ReLU()
        self.fc_out = torch.nn.Linear(fc_hidden_dim, output_dim)
        self.sigmoid = torch.nn.Sigmoid()


    def forward(self, text):
        embedded = self.embedding(text)
        _, (hidden_state, cell_state) = self.lstm(embedded)
        hidden = torch.cat((hidden_state[-2,:,:], hidden_state[-1,:,:]), dim = 1)
        fc_hidden = self.fc_hidden(hidden)
        fc_hidden = self.fc_relu(fc_hidden)
        dense_outputs = self.fc_out(fc_hidden)
        outputs = self.sigmoid(dense_outputs)
        return outputs
    

In [19]:
# Helper functions for the RNN
def get_all_possible_words(vocab, kmer_size=3):
    '''
    Get an exhaustive list of combinations of "AGCT" as kmers of size defined by kmer_size
    '''
    all_com = [''.join(c) for c in product(vocab, repeat=kmer_size)]
    kmer_f_dict = {i + 1: all_com[i] for i in range(0, len(all_com))}
    kmer_r_dict = {all_com[i]: i + 1  for i in range(0, len(all_com))}
    return kmer_f_dict, kmer_r_dict


def convert_seq_2_integers(dataframe, r_dict):
    seq_mat = list()
    for index, item in dataframe.iterrows():
        kmers = item["sequence"].split(" ")
        kmers_integers = [r_dict[k] for k in kmers]
        seq_mat.append(kmers_integers)
    labels = dataframe["label"].tolist()
    return seq_mat, labels

def preprocess_data():
    '''
    Read raw data files and create Pytorch dataset
    '''
    f_dict, r_dict = get_all_possible_words(vocab, 3)
    train_df = pd.read_csv(train_path, sep="\t")
    train_mat, train_labels = convert_seq_2_integers(train_df, r_dict)
    test_df = pd.read_csv(test_path, sep="\t")
    test_mat, test_labels = convert_seq_2_integers(test_df, r_dict)
    return train_mat, train_labels, test_mat, test_labels


class DatasetMaper(Dataset):
	'''
	Handles batches of dataset
	'''
	def __init__(self, x, y):
		self.x = x
		self.y = y
		
	def __len__(self):
		return len(self.x)
		
	def __getitem__(self, idx):
		return self.x[idx], self.y[idx]

In [ ]:
# RNN Evaluation 
def evaluate_model(model, loader_test):
    predictions = []
    model.eval()
    # no gradient update while in test mode
    with torch.no_grad():
        for x_batch, y_batch in loader_test:
            x = x_batch.type(torch.LongTensor)
            y = y_batch.type(torch.FloatTensor)
            y_pred = model(x)
            predictions += list(y_pred.detach().numpy())
    return predictions


def calculate_accuray(ground_truth, predictions):
    true_positives = 0
    true_negatives = 0
    for true, pred in zip(ground_truth, predictions):
        if (pred > 0.5) and (true == 1):
            true_positives += 1
        elif (pred < 0.5) and (true == 0):
            true_negatives += 1
        else:
            pass
    auc_score = np.round(roc_auc_score(ground_truth, predictions), 4)
    accuracy = np.round((true_positives + true_negatives) / len(ground_truth), 4)
    return auc_score, accuracy

In [20]:
# RNN train
def train_model(X_train, y_train, X_test, y_test):
    '''
    Train model using train sequences and their corresponding labels.
    Evaluate trained model using test sequences and their labels
    '''
    model = RNNSeqClassifier()
    print("RNN classifier architecture: \n")
    print(model)
    print()
    X_train = torch.tensor(X_train, dtype=torch.long)
    y_train = torch.tensor(y_train, dtype=torch.long)

    X_test = torch.tensor(X_test, dtype=torch.long)
    y_test = torch.tensor(y_test, dtype=torch.long)

    training_set = DatasetMaper(X_train, y_train)
    test_set = DatasetMaper(X_test, y_test)
		
    loader_training = DataLoader(training_set, batch_size=batch_size)
    loader_test = DataLoader(test_set)

    optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)
    loss_epo = list()
    for epoch in range(n_epochs):
        loss_bat = list()
        predictions = []
        # training mode
        model.train()
        for x_batch, y_batch in loader_training:
            x = x_batch.type(torch.LongTensor)
            y = y_batch.type(torch.FloatTensor)
            y_pred = model(x)
            y_pred = torch.reshape(y_pred, (y_pred.shape[0], ))
            loss = F.binary_cross_entropy(y_pred, y)
            loss_bat.append(loss.detach().numpy())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
        loss_epo.append(np.mean(loss_bat))
        # evaluate model after each epoch of training on test data
        pred = evaluate_model(model, loader_test)
        auc_score, acc = calculate_accuray(y_test, pred)
        mean_batch_loss = np.mean(np.round(loss_bat, 2))
        # output training and evaluation results
        print("Training: Loss at epoch {} : {}".format(epoch+1, np.round(mean_batch_loss, 4)))
        print("Test: Accuracy at epoch {} : {}".format(epoch+1, acc))
        print("Test: ROC AUC score at epoch {} : {}".format(epoch+1, auc_score))
        print()
    print("Training: Loss after {} epochs: {}".format(epoch+1, np.mean(loss_epo)))

The following cell trains and evaluates the RNN using 5-fold cross-validation. The following cell is an alternative to train and evaluate on the full dataset.

In [11]:
# Alternative for RNN to train and evaluate using 5-fold cross-validation
if __name__ == "__main__":
    pos_fasta = "./data/positives.fa"
    neg_fasta = "./data/negatives.fa"

    pos_seqs_dic = read_fasta_into_dic(pos_fasta, 
                                       convert_to_rna=True, 
                                       all_uc=True,
                                       skip_n_seqs=True)
    neg_seqs_dic = read_fasta_into_dic(neg_fasta,
                                       convert_to_rna=True, 
                                       all_uc=True,
                                       skip_n_seqs=True)

    # Create 5 folds
    folds = merge_pos_neg_kfold(pos_seqs_dic, neg_seqs_dic, n_splits=5, k=3)

    
    vocab = ["A", "C", "G", "T"]
    f_dict, r_dict = get_all_possible_words(vocab, kmer_size=3)

    
    all_aucs = []
    all_accs = []

    for i, (train_df, test_df) in enumerate(folds):
        print(f"\n=== FOLD {i+1} / {len(folds)} ===")

        # Convert train_df & test_df sequences into integer matrices
        X_train, y_train = convert_seq_2_integers(train_df, r_dict)
        X_test, y_test = convert_seq_2_integers(test_df, r_dict)

        train_model(X_train, y_train, X_test, y_test)


=== FOLD 1 / 5 ===
RNN classifier architecture: 

RNNSeqClassifier(
  (embedding): Embedding(4097, 128)
  (lstm): LSTM(128, 100, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (fc_hidden): Linear(in_features=200, out_features=64, bias=True)
  (fc_relu): ReLU()
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

Training: Loss at epoch 1 : 0.5496000051498413
Test: Accuracy at epoch 1 : 0.8364
Test: ROC AUC score at epoch 1 : 0.8703

Training: Loss at epoch 2 : 0.5911999940872192
Test: Accuracy at epoch 2 : 0.6406
Test: ROC AUC score at epoch 2 : 0.7235

Training: Loss at epoch 3 : 0.609000027179718
Test: Accuracy at epoch 3 : 0.6841
Test: ROC AUC score at epoch 3 : 0.7543

Training: Loss at epoch 4 : 0.527899980545044
Test: Accuracy at epoch 4 : 0.805
Test: ROC AUC score at epoch 4 : 0.8447

Training: Loss at epoch 5 : 0.4422999918460846
Test: Accuracy at epoch 5 : 0.8396
Test: ROC AUC score at epoch 5 : 0.888

Training: Loss at e

Uncomment the following cell to train and evaluate the RNN on thr full dataset

In [ ]:
# Preprocess the data and then train and evaluate the RNN (without cross-validation)
# X_train, y_train, X_test, y_test = preprocess_data()
# train_model(X_train, y_train, X_test, y_test)

RNN classifier architecture: 

RNNSeqClassifier(
  (embedding): Embedding(4097, 128)
  (lstm): LSTM(128, 100, num_layers=2, batch_first=True, dropout=0.1, bidirectional=True)
  (fc_hidden): Linear(in_features=200, out_features=64, bias=True)
  (fc_relu): ReLU()
  (fc_out): Linear(in_features=64, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

Training: Loss at epoch 1 : 0.5515999794006348
Test: Accuracy at epoch 1 : 0.8308
Test: ROC AUC score at epoch 1 : 0.8789

Training: Loss at epoch 2 : 0.4058000147342682
Test: Accuracy at epoch 2 : 0.8501
Test: ROC AUC score at epoch 2 : 0.8872

Training: Loss at epoch 3 : 0.3767000138759613
Test: Accuracy at epoch 3 : 0.867
Test: ROC AUC score at epoch 3 : 0.9014

Training: Loss at epoch 4 : 0.358599990606308
Test: Accuracy at epoch 4 : 0.884
Test: ROC AUC score at epoch 4 : 0.9122

Training: Loss at epoch 5 : 0.366100013256073
Test: Accuracy at epoch 5 : 0.8453
Test: ROC AUC score at epoch 5 : 0.9125

Training: Loss at epoch 6 : 0.335000008

After training and evaluation, our models achieved the following results:
1) RNN \
Accuracy: 0.8952 \
ROC AUC: 0.9411


2) DNABERT \
Accuracy: 0.9259 \
ROC AUC: 0.9692 


Thus, DNABERT performed better for predicting the binding sites of this protein, outperforming the RNN approach by 2% in accuracy and nearly 3% in ROC AUC. 

BONUS: The current maximum sequence length is set to 100nt. Try increasing this gradually and compare the difference in performance of the two models again. 